## collecting dataSet images

In [17]:
import cv2
import numpy as np

## getting background

In [19]:
def get_background(img,weight_of_input_img=0.5):
    global background
    if(background is None):
        background = img.copy().astype('float')
        return 
    else:
        cv2.accumulateWeighted(img, background, weight_of_input_img)#adding new frame to the background
        return 

In [24]:
background = None
weight_of_input_img = 0.1
num_frames = 1
cap = cv2.VideoCapture(0)
while(cap.isOpened()):
    ret, frame = cap.read()#captureing a frame
    image = cv2.resize(frame,(500,500))#resizing the frame
    image = cv2.flip(image,1)
    img_gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)#converting to grayscale image
    if(num_frames <= 100):
        get_background(img_gray,weight_of_input_img)#getting the background
        num_frames+=1
        img_gray = cv2.putText(img_gray,'Capturing Background',(20,20),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)##putting txt on the image
        cv2.imshow('live_feed',img_gray)#showing the image
        cv2.waitKey(1) & 0xFF
        continue
        
    elif(num_frames == 101):
        background = background.astype('uint8')#converting bsckgroung to unsigned int
        background = cv2.GaussianBlur(background,(5,5),0)#blurring the background
        num_frames+=1
        bg = cv2.putText(background.copy(),'Background Image',(20,20),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)#putting text on the window
        cv2.imshow('background',bg)
        cv2.waitKey(1) & 0xFF
        continue
        
    elif(num_frames > 101):
        img_gray = cv2.putText(img_gray,'Background Captured',(20,20),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)
        cv2.imshow('live_feed',img_gray)
        
    key = cv2.waitKey(1) & 0xFF
    if(key == ord('q') or key == ord('Q')):
        break
cap.release()
cv2.destroyAllWindows()

## getting foreground by background substraction

In [22]:
def get_hand_contour_and_threshold_image(img,background,threshold = 30):
    img_diff_form_background = cv2.absdiff(img, background)#calculating the difference in new frame and background
    
    _,img_threshold = cv2.threshold(img_diff_form_background,threshold,255,cv2.THRESH_BINARY)#getting threshold image
    
    _, contours, _ = cv2.findContours(img_threshold.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)#finding contours in the image
    
    if(len(contours) == 0):#if no contour then we will return
        return
    else:
        contour = max(contours, key = cv2.contourArea)##giving contour with maximun area
        
        return contour, img_threshold #returning the contour with max area and threshold image

In [37]:
roi_background = background[:275,225:]#declaring the background region of interest
threshold_value = 30 #threshold value
start_saving = False #variable that on being true allows to start saving the images 
num_images = 0 # tells the number of images captured
images_to_get = 100
cap = cv2.VideoCapture(0)
while(cap.isOpened()):
    ret,frame = cap.read()
    
    image = cv2.resize(frame,(500,500))
    
    image = cv2.flip(image,1)
    
    img_gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY) #converting to grayscale
    
    img_gray_blur = cv2.GaussianBlur(img_gray,(5,5),0) #blurring the image 
    
    roi_blur_image = img_gray_blur[:275,225:]#deciding the region of interst for blurr image
    
    roi_img_gray = img_gray[:275,225:]#deciding the region of interst for gray scale
    
    hand_contour_and_threshold_image = get_hand_contour_and_threshold_image(roi_blur_image, roi_background, threshold_value)#getting hand contour and threshold image and none is no hand in the box
    
    if(hand_contour_and_threshold_image is not None and num_images!=images_to_get):#if hand_contour_and_threshold_image is None then it means that or hand is not in the box
        
        hand_contour, threshold_image = hand_contour_and_threshold_image #getting the contour with max area and threshold image
        
        x,y,w,h = cv2.boundingRect(hand_contour) #getting the coordinates(x,y) rectangle and height and width (h,w) which inscribe the contour
        
        cv2.rectangle(roi_img_gray,(x,y),(x+w,y+h),(0,255,0),2) #drawing the contour reactange on the roi of grayscale image
        
        cv2.rectangle(img_gray,(220,0),(500,280),(255,0,0),2) #drawing the rectangle in which we have to place our hand to capture and store image
        
        cv2.imshow('dg',img_gray)
        
        cv2.imshow('threshold',threshold_image)
        
        if(start_saving == False):
            signal_to_start_saving = cv2.waitKey(1) & 0xFF #getting a key from keyboard to start the saving of images
            
            if(signal_to_start_saving == ord('s') or signal_to_start_saving == ord('S')):#turns the start_saving var true if we press S or s on keyboard
                start_saving = True
        else:
            location_to_save_image = r'F:\project hand gestures\hand dataset\test dataset\hand rock' #location of saving the image
            
            cv2.imwrite(location_to_save_image +'\hand_rock'+str(num_images)+'.jpg',threshold_image)#saving the image to a location
            
            num_images+=1 #increasing the number of images after saving one image
    else: # this means that hand is not in the box or we have reached specified number of images
        start_saving = False #stop saving images by making this false
        if(num_images >= images_to_get):
                cv2.putText(img_gray,'dataset taken',(100,100),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)#putting text on the window telling that dataset is taken
        else:
            cv2.putText(img_gray,'Place hand in box',(100,100),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)#putting text on the window to place hand in the box to start saving the image

        cv2.rectangle(img_gray,(220,0),(500,280),(255,0,0),2)#drwaing rectangle to place hand in
        
        cv2.imshow('dg',img_gray)
    
    key = cv2.waitKey(1) & 0xFF
    if(key == ord('q') or key == ord('Q')):
        break
cap.release()
cv2.destroyAllWindows()